In [2]:
import calendar
import datetime
from datetime import timedelta ,datetime
startDate = datetime.strptime("20220723", "%Y%m%d")

date = startDate + timedelta(days=1)
year= date.year
month = date.month
day = date.day

#系统默认：星期一作为一周的第一天（即：0），星期日作为一周的最后一天（即：6）
print(calendar.weekday(year,month,day))
currentday =calendar.weekday(year,month,day)


6


In [6]:
import akshare as ak
from datetime import date,datetime
print(date.today().strftime("%Y%m%d"))

20220724


In [1]:
import sys
sys.path.append(".")
import utils.mongodbUtil as mongodbUtil

# mongodbUtil.getClient("stock")


from pyecharts.charts import Bar
from pyecharts import options as opts

# V1 版本开始支持链式调用
bar = (
    Bar()
    .add_xaxis(["衬衫", "毛衣", "领带", "裤子", "风衣", "高跟鞋", "袜子"])
    .add_yaxis("商家A", [114, 55, 27, 101, 125, 27, 105])
    .add_yaxis("商家B", [57, 134, 137, 129, 145, 60, 49])
    .set_global_opts(title_opts=opts.TitleOpts(title="某商场销售情况"))
)
bar.render_notebook()

# 不习惯链式调用的开发者依旧可以单独调用方法
bar = Bar()
bar.add_xaxis(["衬衫", "毛衣", "领带", "裤子", "风衣", "高跟鞋", "袜子"])
bar.add_yaxis("商家A", [114, 55, 27, 101, 125, 27, 105])
bar.add_yaxis("商家B", [57, 134, 137, 129, 145, 60, 49])
bar.set_global_opts(title_opts=opts.TitleOpts(title="某商场销售情况"))
bar.render_notebook()

In [8]:
# 导入库包
import pyecharts.options as opts
from pyecharts.charts import Line

# 写入数据
years = ["1995", "1996", "1997", "1998", "1999", "2000",
         "2001", "2002", "2003", "2004", "2005", "2006",
         "2007", "2008", "2009"]
Postage = [0.32, 0.32, 0.32, 0.32, 0.33, 0.33, 0.34,
           0.37, 0.37, 0.37, 0.37, 0.39, 0.41, 0.42, 0.44]

(
    Line()
    # 全局配置
    .set_global_opts()
    # x轴
    .add_xaxis(xaxis_data=years)
    # y轴
    .add_yaxis(
        series_name="",         # 数据所属集合的名称
        y_axis=Postage,            # y轴数据
    )
    # 输出为HTML文件
    .render_notebook()
)

In [13]:
import sys
sys.path.append(".")
import utils.mongodbUtil as mongodbUtil
from  datetime import datetime
gdpCol = mongodbUtil.getCol("stock","gdp")
dailyCol = mongodbUtil.getCol("stock","daily_deal")


def  getGdpDate(year, quarter):
    if quarter == 0:
        return str(year-1)+"-12-01"
    if quarter ==4:
        return str(year)+'-12-01'
    return str(year)+'-0'+ str(quarter*3)+'-01'

gdpCursor = gdpCol.find({"date":{"$gt":"2019-03-01"}}).sort("date",1)
gdpDict={}
for gdpData in gdpCursor:
    gdpDict[gdpData["date"]]=gdpData


pipeline = [
    {'$match': {"type": {"$in": ["sh", "sz"]}}},
    {'$group': {'_id': "$date", 'market_amount': {'$sum': "$market_amount"},'deal_amount': {'$sum': "$deal_amount"}}},
    {"$sort": {"_id": 1}}
]
date=[]
amountGdpRate=[]
dailyCursor = dailyCol.aggregate(pipeline)
for dailyData in dailyCursor:
    date.append(dailyData["_id"])
    dt = datetime.strptime(dailyData["_id"], "%Y%m%d") 
    quarter = int(dt.month/3)
    gdp = gdpDict[getGdpDate(dt.year,quarter)]['gdp']
    amountGdpRate.append(dailyData['market_amount']/gdp)

import pyecharts.options as opts
from pyecharts.charts import Line

(
    Line()
    # 全局配置
    .set_global_opts()
    # x轴
    .add_xaxis(xaxis_data=date)
    # y轴
    .add_yaxis(
        series_name="市值gdp比值",         # 数据所属集合的名称
        y_axis=amountGdpRate,            # y轴数据
    )
    # 输出为HTML文件
    .render_notebook()
)